# Tree Methods

## Imported Libraries

In [1]:
import pandas as pd
import numpy as np

## Data Extraction

In [2]:
trainingData = pd.read_csv("../Data/train.csv")
trainingData[trainingData['Survived'] == 1]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...
875,876,1,3,"Najib, Miss. Adele Kiamie ""Jane""",female,15.0,0,0,2667,7.2250,NaN,C
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


## Tree Data Structure

In [3]:
class Tree:
    
    def __init__(self, feature, children):
        self.feature = feature
        self.children = children
        
    def getFeature(self):
        return self.feature
    
    def search(self,datapoint):
        if self.children is None:
            return self.getFeature()
        point = datapoint.get(self.feature)
        for child in self.children:
            if child[0] == point:
                return child[1].search(datapoint)
        

## Tree Structure Test

In [4]:
leaf1 = Tree(0, None)
leaf2 = Tree(1, None)
leaf3 = Tree(1, None)

branch = Tree("test2", [["b1",leaf1], ["b2",leaf2]])
root = Tree("test1", [["a1",branch], ["a2",leaf3]])

EXdata = pd.Series(data=["a1","b1"],index=["test1","test2"])

assert 0 == root.search(EXdata)

## Information Gain Functions

In [5]:
def Entropy(dataSet):
    result1 = dataSet[dataSet['Survived'] == 1].size/dataSet.size
    result1 = result1*np.log2(result1)
    result2 = dataSet[dataSet['Survived'] == 0].size/dataSet.size
    result2 =  result2*np.log2(result2)
    return -1* (result1 + result2)

0.9607079018756469


In [7]:
def InformationGain(dataSet, feature):
    
    totalE = Entropy(dataSet)
    sumE = 0
    
    if feature in ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]:
    
        values, counts = np.unique(dataSet[feature], return_counts = True)
        for val in values:
            sumE += (dataSet[dataSet[feature] == val].size)/dataSet.size*Entropy(dataSet[dataSet[feature] == val])
        return totalE - sumE
    
    elif feature in ["Age", "Ticket", "Fare"]:
        
        
        return 0
    
    else:
        
        return 0
        
        

print(InformationGain(trainingData, "Pclass"))

0.0838310452960116
